In [51]:
import chromadb
from sentence_transformers import SentenceTransformer, CrossEncoder
from langchain_anthropic import ChatAnthropic
import numpy as np
from typing import List, Dict
import os
from IPython.display import display, Markdown

In [52]:
class StreamlinedAdvancedRAG:
    """
    Advanced RAG ແບບງ່າຍ - ແອັດໃຊ້ Process ນີ້ເປັນຫຼັກ:
    1. Dense Retrieval 
    2. Query Rewriting (LLM)
    3. HyDE (LLM)
    4. Re-ranking
    """
    
    def __init__(self, collection_name: str, anthropic_api_key: str = None):
        # ຕິດຕັ້ງພື້ນຖານ
        self.client = chromadb.PersistentClient(path="../Vector/chroma_db")
        self.collection = self._load_collection(collection_name)
        
        # Models
        self.embedding_model = SentenceTransformer('D:/model/BAAI-bge-m3',device='cpu')
        self.rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
        # LLM Setup
        api_key = anthropic_api_key or os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            raise ValueError("ກະລຸນາໃຫ້ ANTHROPIC_API_KEY")
        
        # Main LLM (Claude Opus 4)
        self.llm = ChatAnthropic(
            api_key=api_key,
            model="claude-opus-4-1-20250805",
            temperature=0.1,
            max_tokens=2000
        )
        
        # Query processing LLM (ໃຊ້ Sonnet ເພື່ອປະຫຍັດ)
        self.query_llm = ChatAnthropic(
            api_key=api_key,
            model="claude-3-7-sonnet-20250219",
            temperature=0.0,
            max_tokens=300
        )
        
        print(f"✅ Streamlined Advanced RAG ready! ({self.collection.count()} documents)")
    
    def _load_collection(self, collection_name: str):
        """ໂຫຼດ collection"""
        try:
            return self.client.get_collection(name=collection_name)
        except Exception as e:
            raise ValueError(f"Cannot load collection '{collection_name}': {e}")
    
    # ==================== 1. QUERY REWRITING ====================
    
    def rewrite_query(self, query: str) -> str:
        """
        ໃຊ້ LLM ປັບປຸງຄໍາຖາມໃຫ້ເໝາະສົມກັບການຄົ້ນຫາ
        """
        rewriting_prompt = f"""ທ່ານເປັນຜູ້ຊ່ວຍໃນການປັບປຸງຄໍາຖາມສໍາລັບລະບົບຄົ້ນຫາເອກະສານ.

ຄໍາຖາມຕົ້ນສະບັບ: "{query}"

ກະລຸນາປັບປຸງຄໍາຖາມນີ້ໃຫ້:
- ເພີ່ມຄວາມໝາຍໃຫ້ຈະແຈ້ງ ແລະ ລະອຽດ
- ເພີ່ມຄໍາສໍາຄັນທີ່ກ່ຽວຂ້ອງ
- ປັບໃຊ້ໂຄງສ້າງທີ່ເຫມາະສົມ
- ສັ້ນແລະກະຊັບ

ຕອບແຕ່ຄໍາຖາມທີ່ປັບປຸງແລ້ວເທົ່ານັ້ນ:"""

        try:
            response = self.query_llm.invoke(rewriting_prompt)
            rewritten = response.content.strip()
            
            # ເອົາ quotes ອອກຖ້າມີ
            rewritten = rewritten.strip('"\'')
            
            print(f"🔄 Query rewritten: '{query}' → '{rewritten}'")
            return rewritten
            
        except Exception as e:
            print(f"⚠️  Query rewriting failed: {e}")
            return query
    
    # ==================== 2. HyDE GENERATION ====================
    
    def generate_hyde(self, query: str) -> str:
        """
        ສ້າງເອກະສານສົມມຸດຕິຖານດ້ວຍ Claude Opus 4
        """
        hyde_prompt = f"""ທ່ານເປັນຜູ້ຊ່ວຍທີ່ຊ່ຽວຊານໃນດ້ານການສ້າງເອກະສານສົມມຸດຕິຖານ

ຄໍາຖາມ: {query}

ຂຽນເອກະສານ:
- ຕອບຄໍາຖາມໂດຍກົງແລະຊັດເຈນ
- ໃຊ້ພາສາແລະຄໍາສັບທີ່ມີໂອກາດປາກົດໃນເອກະສານຈິງ
- ມີເນື້ອຫາທີ່ມີຄຸນນະພາບແລະຖືກຕ້ອງ
- ຍາວປະມານ 100-150 ຄໍາ
- ເຂົ້າເລື່ອງໂດຍກົງ ບໍ່ຕ້ອງມີບົດນໍາ

ເອກະສານ:"""
        
        try:
            response = self.llm.invoke(hyde_prompt)
            hyde_doc = response.content.strip()
            
            print(f"📝 HyDE generated: {len(hyde_doc)} chars")

            return hyde_doc
            
        except Exception as e:
            print(f"⚠️  HyDE generation failed: {e}")
            return query
    
    # ==================== 3. DENSE RETRIEVAL ====================
    
    def dense_retrieval(self, query: str, n_results: int = 10) -> List[Dict]:
        """
        Dense retrieval ດ້ວຍ semantic embeddings
        """
        query_embedding = self.embedding_model.encode([query]).tolist()
        
        results = self.collection.query(
            query_embeddings=query_embedding,
            n_results=n_results
        )
        
        docs = []
        for i in range(len(results['documents'][0])):
            docs.append({
                'text': results['documents'][0][i],
                'score': 1 - results['distances'][0][i],  # Convert distance to similarity
                'id': results['ids'][0][i]
            })
        
        return docs
    
    # ==================== 4. RE-RANKING ====================
    
    def rerank_documents(self, query: str, docs: List[Dict], top_k: int = 5) -> List[Dict]:
        """
        Re-ranking ດ້ວຍ cross-encoder
        """
        if len(docs) <= top_k:
            return docs
        
        print(f"🔄 Re-ranking {len(docs)} → {top_k} documents...")
        
        try:
            # ສ້າງ query-document pairs
            pairs = [(query, doc['text']) for doc in docs]
            
            # ຄິດໄລ່ relevance scores
            scores = self.rerank_model.predict(pairs)
            
            # ເພີ່ມ rerank scores
            for i, doc in enumerate(docs):
                doc['rerank_score'] = float(scores[i])
                doc['original_rank'] = i + 1
            
            # Sort ໂດຍ rerank score
            reranked = sorted(docs, key=lambda x: x['rerank_score'], reverse=True)
            
            print(f"✅ Re-ranking completed")
            return reranked[:top_k]
            
        except Exception as e:
            print(f"⚠️  Re-ranking failed: {e}")
            return docs[:top_k]
    
    # ==================== MAIN SEARCH PIPELINE ====================
    
    def search_advanced(self, query: str,
                       n_results: int = 10, 
                       use_rewriting: bool = True,
                       use_hyde: bool = True,
                       use_reranking: bool = True,
                       top_k: int = 5
                       ) -> List[Dict]:
        """
        Advanced search pipeline ແບບງ່າຍ
        """
        print(f"🚀 Advanced search: '{query}'")
        
        search_queries = [query]  # ເລີ່ມດ້ວຍຄໍາຖາມຕົ້ນສະບັບ

        # 1. Query Rewriting
        if use_rewriting:
            rewritten = self.rewrite_query(query)
            if rewritten != query:
                search_queries.append(rewritten)
        
        # 2. HyDE
        if use_hyde:
            hyde_doc = self.generate_hyde(query)
            search_queries.append(hyde_doc)
        
        # 3. Dense Retrieval ສໍາລັບແຕ່ລະ query
        all_docs = {}
        for i, sq in enumerate(search_queries):
            print(f"🔍 Searching with query {i+1}/{len(search_queries)}")

            docs = self.dense_retrieval(sq, n_results)
            
            # ລວມຜົນໄດ້ຮັບ (ຖ້າເອກະສານຊ້ໍາ ຈະລວມ scores)
            for doc in docs:
                doc_id = doc['id']
                if doc_id in all_docs:
                    # ລວມ scores ແລະ ເພີ່ມ weight
                    all_docs[doc_id]['score'] = max(all_docs[doc_id]['score'], doc['score'])
                    all_docs[doc_id]['query_count'] = all_docs[doc_id].get('query_count', 1) + 1
                else:
                    doc['query_count'] = 1
                    all_docs[doc_id] = doc
        
        # Convert to list and sort by score
        retrieved_docs = sorted(all_docs.values(), key=lambda x: x['score'], reverse=True)
        retrieved_docs = retrieved_docs[:n_results]
        
        print(f"📄 Retrieved {len(retrieved_docs)} unique documents")
        
        # 4. Re-ranking
        if use_reranking and len(retrieved_docs) > 3:
            final_docs = self.rerank_documents(query, retrieved_docs, 
                                             min(n_results, len(retrieved_docs),top_k))
        else:
            final_docs = retrieved_docs
        
        return final_docs
    
    # ==================== MAIN Q&A FUNCTION ====================
    
    def ask(self, question: str, n_results: int = 8, **kwargs) -> Dict:
        """
        ຖາມຄໍາຖາມດ້ວຍ Streamlined Advanced RAG
        """
        print(f"\n{'='*60}")
        print(f"❓ ຄໍາຖາມ: {question}")
        print(f"{'='*60}")
        
        # ຄົ້ນຫາເອກະສານ
        docs = self.search_advanced(question, n_results, **kwargs)
        
        if not docs:
            return {"error": "ບໍ່ພົບເອກະສານທີ່ກ່ຽວຂ້ອງ"}
        
        # ສ້າງ context
        context_parts = []
        for i, doc in enumerate(docs, 1):
            score_info = f"(Score: {doc.get('rerank_score', doc['score']):.3f})"
            context_parts.append(f"[Document {i}] {score_info}\n{doc['text']}")
        
        context = "\n\n".join(context_parts)
        
        # ສ້າງ prompt
        prompt = f"""ທ່ານເປັນຜູ້ຊ່ວຍ AI ທີ່ຊ່ຽວຊານໃນການຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາ.

        ຄຳແນະນຳ:
        1. ຕອບຄຳຖາມໂດຍອ້າງອີງຈາກເອກະສານທີ່ໃຫ້ມາເທົ່ານັ້ນ
        2. ຖ້າບໍ່ພົບຄຳຕອບໃນເອກະສານ, ໃຫ້ບອກວ່າບໍ່ພົບຂໍ້ມູນທີ່ກ່ຽວຂ້ອງ
        3. ລະບຸແຫຼ່ງຂໍ້ມູນທີ່ໃຊ້ໃນການຕອບ
        4. ຕອບເປັນພາສາລາວເທົ່ານັ້ນ, ໃຫ້ຄຳຕອບທີ່ຊັດເຈນ ແລະ ລະອຽດ
        5. ຕອບໃຫ້ເປັນ Format markdown
                
        ຂໍ້ມູນອ້າງອີງ:
        {context}

        ຄໍາຖາມ: {question}
        """
        
        # ສ້າງຄໍາຕອບ
        try:
            response = self.llm.invoke(prompt)
            answer = response.content.strip()
            
            return {
                'question': question,
                'answer': answer,
                'sources': docs,
                'metadata': {
                    'total_sources': len(docs),
                    'avg_score': np.mean([doc.get('rerank_score', doc['score']) for doc in docs]),
                    'features_used': {
                        'query_rewriting': kwargs.get('use_rewriting', True),
                        'hyde': kwargs.get('use_hyde', True),
                        'reranking': kwargs.get('use_reranking', True)
                    }
                }
            }
            
        except Exception as e:
            return {"error": f"ເກີດຂໍ້ຜິດພາດໃນການສ້າງຄໍາຕອບ: {str(e)}"}

In [53]:
# ==================== MAIN USAGE ====================

def main():
    """
    ຕົວຢ່າງການໃຊ້ງານ
    """
    # ສ້າງ RAG system
    rag = StreamlinedAdvancedRAG(
        collection_name="pdf_documents",
        anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
    ) 
    
    # ຖາມຄໍາຖາມ
    result = rag.ask(
        question="SMS Banking Package ສະໝັກແນວໃດ ແລະ ມີ Package ຍັງແນ່?",
        n_results=20, # ຈຳນວນເອກະສານທີ່ຈະຄົ້ນຫາ
        use_rewriting=True,    # ໃຊ້ LLM ປັບປຸງຄໍາຖາມ  True = ເປີດ / False = ປິດ
        use_hyde=True,        # ບໍ່ໃຊ້ HyDE  True = ເປີດ / False = ປິດ
        use_reranking=True,   # ບໍ່ໃຊ້ re-ranking  True = ເປີດ / False = ປິດ
        top_k=10 # ຈຳນວນ Ranking ທີ່ຈະຄົ້ນຫາ
    )
    
    if 'error' not in result:
        display(Markdown(f"\n✅ ຄໍາຕອບ: {result['answer']}"))   
        display(Markdown(f"\n📊 ຂໍ້ມູນ:")) 
        display(Markdown(f"   Sources: {result['metadata']['total_sources']}")) 
        display(Markdown(f"   Avg Score: {result['metadata']['avg_score']:.3f}")) 
        display(Markdown(f"   Features: {result['metadata']['features_used']}"))  
    else:
        print(f"\n❌ {result['error']}")
        

In [54]:
if __name__ == "__main__":
    main()

✅ Streamlined Advanced RAG ready! (95 documents)

❓ ຄໍາຖາມ: SMS Banking Package ສະໝັກແນວໃດ ແລະ ມີ Package ຍັງແນ່?
🚀 Advanced search: 'SMS Banking Package ສະໝັກແນວໃດ ແລະ ມີ Package ຍັງແນ່?'
🔄 Query rewritten: 'SMS Banking Package ສະໝັກແນວໃດ ແລະ ມີ Package ຍັງແນ່?' → 'ວິທີສະໝັກນຳໃຊ້ບໍລິການ SMS Banking ຂອງທະນາຄານ ແລະ ມີແພັກເກັດໃດແດ່ທີ່ສາມາດເລືອກໄດ້?'
📝 HyDE generated: 619 chars
🔍 Searching with query 1/3
🔍 Searching with query 2/3
🔍 Searching with query 3/3
📄 Retrieved 20 unique documents
🔄 Re-ranking 20 → 10 documents...
✅ Re-ranking completed



✅ ຄໍາຕອບ: ອີງຕາມເອກະສານທີ່ໃຫ້ມາ, ຂ້ອຍຂໍຕອບກ່ຽວກັບ SMS Banking Package ດັ່ງນີ້:

## ວິທີການສະໝັກ SMS Banking Package:

ທ່ານສາມາດສະໝັກ Package ເພີ່ມໄດ້ໂດຍ:
- **ພິມ B10 [ຍະຫວ່າງ] <ເລກບັນຊີ> ແລ້ວສົ່ງເບີ 1444**

## Package ທີ່ມີໃຫ້ເລືອກ:

| ລະຫັດ Package | ຈຳນວນເງິນ (ກີບ) | ຈຳນວນວັນທີ່ໃຊ້ໄດ້ | ຈຳນວນຂໍ້ຄວາມ |
|--------------|----------------|-----------------|--------------|
| **B05** | 5,000 | 365 ວັນ | 30 ຂໍ້ຄວາມ |
| **B10** | 10,000 | 365 ວັນ | 60 ຂໍ້ຄວາມ |
| **B25** | 25,000 | 365 ວັນ | 150 ຂໍ້ຄວາມ |
| **B50** | 50,000 | 365 ວັນ | 300 ຂໍ້ຄວາມ |
| **B100** | 100,000 | 365 ວັນ | 600 ຂໍ້ຄວາມ |

## ຟັງຊັນອື່ນໆທີ່ເປັນປະໂຫຍດ:
- ກວດ Package ປັດຈຸບັນ: ພິມ **C [ຍະຫວ່າງ] <ເລກບັນຊີ>** ແລ້ວສົ່ງເບີ 1444
- ທຸກ Package ສາມາດໃຊ້ໄດ້ 365 ວັນ (1 ປີເຕັມ)

**ແຫຼ່ງຂໍ້ມູນ:** Document 3 - ປື້ມສັງລວມຜະລິດຕະພັນທັງໝົດຂອງ ທຄຕລ_2020_Update.2 ໜ້າທີ 15


📊 ຂໍ້ມູນ:

   Sources: 10

   Avg Score: 4.395

   Features: {'query_rewriting': True, 'hyde': True, 'reranking': True}